In [ ]:
!pip install transformers
!pip install jsonlines
!pip install spacy
!pip install yake

     |████████████████████████████████| 3.1 MB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.0 MB/s 
     |████████████████████████████████| 59 kB 9.9 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
     |████████████████████████████████| 895 kB 89.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 60 kB 4.9 MB/s 
     |████████████████████████████████| 137 kB 18.2 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.8.9-cp37-cp37m-linux_x86_64.whl size=73238 sha256=992e436f3ffacf4a6d7e8e669f8277452bab24b8ca2eceb67ac88e9f2727780f
  Stored in directory: /root/.cache/pip/wheels/32/a9/ef/5d8742e72deaf0d1de327a180d008c2c0299367581800ea73f
  Created wheel for segtok: filename=segtok-1.5.10-py3-none-any.whl size=25030 sha256=7625de9268f8baf0650e2b5c078a24163d6d63cdedeb79891bbcb7c173c9dfd6
  Stored in directory: /r

In [ ]:
import jsonlines
import pandas as pd
import numpy as np
import yake
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from gensim.models import Word2Vec
from transformers import pipeline
import torch
import tensorflow as tf
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def cleanupDoc(s):
    # Remove stopwords
    stopset = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(s)
    
    # Remove non alpha numeric words
    tokens_clean = [word for word in tokens if word.isalnum()]
    
    # Lemmatize 
    lemmatizer = WordNetLemmatizer()
    tokens_lemma = [lemmatizer.lemmatize(word) for word in tokens]
    
    cleanup = " ".join(filter(lambda word: word not in stopset, s.split()))
    return cleanup

In [ ]:
# Load in the cs142 messages and keywords
corpus142 = []

with jsonlines.open('/content/drive/MyDrive/Programming/DiscordData/cs142.jsonl') as f:
    for line in f.iter():
        corpus142 += line['text'].split()
        
corpus142_clean = map(cleanupDoc, corpus142)
document142 = [" ".join(corpus142_clean)]
        
kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 1
deduplication_threshold = 0.9
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(document142[0])
for kw in keywords:
    print(kw)

('lab', 0.0003622927497728561)
('yeah', 0.00044409286151934305)
('function', 0.000509230457218628)
('code', 0.0008595546968200261)
('class', 0.0009008227432156448)
('functions', 0.0010781363586425642)
('make', 0.0012229268962260422)
('work', 0.00180260227264513)
('zybooks', 0.0018164189977516768)
('thing', 0.0019021729403232088)
('labs', 0.002071106886201494)
('time', 0.0021715056992513766)
('lol', 0.002200995060084047)
('good', 0.0022901153535326555)
('Vector', 0.0023153509655184307)
('error', 0.0028438104010345094)
('string', 0.002976103074024418)
('int', 0.0030467327247291794)
('main', 0.0031490502457447887)
('loop', 0.0034186297318767782)


In [ ]:
keywords142 = []
for kw in keywords:
    keywords142.append(kw[0])

In [ ]:
# Reformat so each comment is a "sentence"
comments142 = []
with jsonlines.open('/content/drive/MyDrive/Programming/DiscordData/cs142.jsonl') as f:
    for line in f.iter():
        clean_line = cleanupDoc(line['text'])
        comments142.append(clean_line.split())
print(comments142[10])

['like', 'want', 'something', 'input', 'equal', '142']


In [ ]:
# Strategy 1 -  Train embeddings
model = Word2Vec(comments142, min_count=1)

In [ ]:
# Check things out
for kw in keywords142:
    print(kw, ':')
    print(model.wv.most_similar(kw.lower(), topn=5), '\n')
    
# Overall results are ok - we'd need to do a lot more filtering 

lab :
[('I', 0.9992942810058594), ('use', 0.9990569353103638), ('would', 0.9990178942680359), ('need', 0.9989522099494934), ('function', 0.9989497661590576)] 

yeah :
[('I', 0.9968709945678711), ('make', 0.9968122243881226), ('get', 0.9967653751373291), ('=', 0.9964629411697388), ('code', 0.9964215755462646)] 

function :
[('I', 0.9994499683380127), ('code', 0.9991803765296936), ('make', 0.9991289377212524), ('get', 0.9991282820701599), ('one', 0.9990665912628174)] 

code :
[('I', 0.999281108379364), ('function', 0.9991803169250488), ('get', 0.9990839958190918), ('like', 0.9990746974945068), ('use', 0.9990009069442749)] 

class :
[('I', 0.9990533590316772), ('like', 0.9988999962806702), ('one', 0.9988622069358826), ('function', 0.9987423419952393), ('code', 0.9987059831619263)] 

functions :
[('code', 0.99861741065979), ('I', 0.9985435009002686), ('something', 0.9984685182571411), ("I'm", 0.9984502792358398), ('function', 0.9984447360038757)] 

make :
[('I', 0.9992380142211914), ('func

In [ ]:
# Strategy 2 - BERT

# Get the data again but don't clean
comments142 = []
with jsonlines.open('/content/drive/MyDrive/Programming/DiscordData/cs142.jsonl') as f:
    for line in f.iter():
        comments142.append(line['text'].split())

# Get all the messages that include the keyword to create a paragraph to send to BERT
def get_usages(keyword, corpus):
    usages_paragraph = ""
    for line in corpus:
        if keyword in line:
            usages_paragraph += " ".join(line) + '. '
    return usages_paragraph 

In [ ]:
qa_pipeline = pipeline(
    "question-answering",
    model="csarron/bert-base-uncased-squad-v1",
    tokenizer="csarron/bert-base-uncased-squad-v1"
)

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def ask_bert(keyword, context):
    predictions = qa_pipeline({
        'context': context,
        'question': "What does " + keyword + " mean?"
    })
    return predictions

In [ ]:
for kw in keywords142:
    context = get_usages(kw.lower(), comments142)
    pred = ask_bert(kw.lower(), context)
    print("What is:", kw, "? A:", pred)
    
# Not great either - answers don't seem to be clear enough in the text

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


What is: lab ? A: {'score': 0.8310913443565369, 'start': 10211, 'end': 10228, 'answer': 'song and playlist'}
What is: yeah ? A: {'score': 0.7751052379608154, 'start': 3441, 'end': 3446, 'answer': 'weird'}
What is: function ? A: {'score': 0.586974561214447, 'start': 2452, 'end': 2463, 'answer': 'menu option'}
What is: code ? A: {'score': 0.5887954831123352, 'start': 3078, 'end': 3083, 'answer': 'weird'}
What is: class ? A: {'score': 0.9741919040679932, 'start': 7998, 'end': 8008, 'answer': 'DataAccess'}
What is: functions ? A: {'score': 0.5682612061500549, 'start': 9576, 'end': 9591, 'answer': 'magic functions'}
What is: make ? A: {'score': 0.7020188570022583, 'start': 9555, 'end': 9585, 'answer': 'make sure it returns something'}
What is: work ? A: {'score': 0.1659919023513794, 'start': 2536, 'end': 2542, 'answer': 'coding'}
What is: zybooks ? A: {'score': 0.3072212338447571, 'start': 5016, 'end': 5023, 'answer': 'styling'}
What is: thing ? A: {'score': 0.3490094840526581, 'start': 647

In [ ]:
# Strategy 3 - Train word embedding with MORE DATA!
# Reformat so each comment is a "sentence"
comments = []
for filename in os.listdir('/content/drive/MyDrive/Programming/DiscordData/'):
  print(filename)
  with jsonlines.open('/content/drive/MyDrive/Programming/DiscordData/' + filename) as f:
      for line in f.iter():
          clean_line = cleanupDoc(line['text'])
          comments.append(clean_line.split())
print(comments[10])

byu-cs.jsonl
cs465.jsonl
cs452.jsonl
cs404.jsonl
cs203.jsonl
cs235.jsonl
cs202.jsonl
cs345.jsonl
cs240.jsonl
cs142.jsonl
cs340.jsonl
cs236.jsonl
cs355.jsonl
cs455.jsonl
cs356.jsonl
cs260.jsonl
cs204.jsonl
cs330.jsonl
cs456.jsonl
cs312.jsonl
cs460.jsonl
cs393.jsonl
cs324.jsonl
cs224.jsonl
cs474.jsonl
cs252.jsonl
cs428.jsonl
cs329.jsonl
['Sweet']


In [ ]:
# Train model
model = Word2Vec(comments, min_count=1)

In [ ]:
# Check things out
for kw in keywords142:
    print(kw, ':')
    print(model.wv.most_similar(kw.lower(), topn=5), '\n')

# A little better 

lab :
[('day', 0.9577786922454834), ('today', 0.9571152329444885), ('labs', 0.9550932049751282), ('half', 0.953583836555481), ('homework', 0.9509797096252441)] 

yeah :
[('yeah,', 0.985580325126648), ('yea', 0.9777647852897644), ('shoot', 0.973025381565094), ('wow', 0.9682697057723999), ('wow,', 0.968076229095459)] 

function :
[('file', 0.9774895906448364), ('variable', 0.9632170796394348), ('command', 0.9601997137069702), ('files', 0.9599777460098267), ('object', 0.9587525725364685)] 

code :
[('program', 0.9047635197639465), ('server', 0.8992577791213989), ('compile,', 0.8991215229034424), ('android', 0.8974134922027588), ('app', 0.8973429203033447)] 

class :
[('classes', 0.9710689783096313), ('240', 0.9653234481811523), ('236', 0.9637112617492676), ('labs', 0.9631770253181458), ('major', 0.960992157459259)] 

functions :
[('method', 0.99271160364151), ('files', 0.9905821084976196), ('loop', 0.9880220293998718), ('methods', 0.9877645969390869), ('variables', 0.986646294593811)] 

m